In [21]:
import numpy as np
import pandas as pd

import os
import glob

import ntpath

import matplotlib.pyplot as plt

In [5]:
num_samples = 50
num_classes = 1000
num_images = num_samples * num_classes
class_inds = np.zeros((num_classes, 2))
for j, i in enumerate(range(0, num_images, num_samples)):
    class_inds[j, :] = [i, i + num_samples]

class ResultSummary():
    name = None
    top1_predictions = None
    top5_predictions = None
    test_index = 0
    
    def __init__(self, name, num_tests):
        self.name = name
        self.top1_predictions = np.zeros((num_images, num_tests))
        self.top5_predictions = np.zeros((num_images, num_tests))

    def add_prediction(self, predictions):
        self.top1_predictions[:, self.test_index] = predictions[:, 0]
        self.top5_predictions[:, self.test_index] = predictions[:, 1]
        self.test_index += 1

In [42]:
network_names = [
    'vgg16', 'vgg19',
    'densenet121', 'densenet169', 'densenet201',
    'mobilenet', 'mobilenet_v2',
    'nasnetmobile', 'nasnetlarge',
    'resnet50', 'inception_v3', 'inception_resnet_v2', 'xception',
]
network_names = ['resnet50', 'nasnetlarge']

input_folder = '/home/arash/Desktop/KerasPredictions/'
output_folder = '/home/arash/Desktop/KerasResults/'
if not os.path.exists(output_folder):
    os.mkdir(output_folder)

for experiment_input in glob.glob(input_folder + '/KerasChromacity/'):
    network_predictions = {}
    test_names= []

    experiment_name = experiment_input.split('/')[-2]
    experiment_output = output_folder + '/' + experiment_name + '/'
    if not os.path.exists(experiment_output):
        os.mkdir(experiment_output)
    for net_id, current_network in enumerate(network_names):
        csv_files = sorted(glob.glob(experiment_input + '/' + current_network + '/*.csv'))
        network_predictions[current_network] = ResultSummary(current_network, len(csv_files))
        for file in csv_files:
            if net_id == 0:
                test_name = ntpath.basename(file)
                test_name = test_name.replace(current_network, '')
                test_names.append(test_name.replace('.csv', ''))
            predictions = np.loadtxt(file, delimiter=',')
            network_predictions[current_network].add_prediction(predictions)
    for test_id in range(len(csv_files)):
        (pairwise_comparison1, pairwise_comparison5, pairwise_comparison_classes1, pairwise_comparison_classes5) = report_correlation(network_predictions, test_id)
        output_file = experiment_output + '/top1_corr' + test_names[test_id] + '.csv'
        np.savetxt(output_file, pairwise_comparison1, delimiter=',')
        output_file = experiment_output + '/top5_corr' + test_names[test_id] + '.csv'
        np.savetxt(output_file, pairwise_comparison5, delimiter=',')
        if len(network_names) == 2:
            output_file = experiment_output + '/top1_corr_class' + test_names[test_id] + '.csv'
            np.savetxt(output_file, pairwise_comparison_classes1[0,1,:].flatten(), delimiter=',')
            output_file = experiment_output + '/top5_corr_class' + test_names[test_id] + '.csv'
            np.savetxt(output_file, pairwise_comparison_classes5[0,1,:].flatten(), delimiter=',')

In [35]:
num_networks = len(network_names)
def report_correlation(network_predictions, test_id):
    pairwise_comparison1 = np.ones((num_networks, num_networks))
    pairwise_comparison5 = np.ones((num_networks, num_networks))
    pairwise_comparison_classes1 = np.ones((num_networks, num_networks, num_classes))
    pairwise_comparison_classes5 = np.ones((num_networks, num_networks, num_classes))
    for i in range(num_networks - 1):
        name_i = network_names[i]
        pred_i = network_predictions[name_i].top1_predictions[:, test_id]
        pred_i5 = network_predictions[name_i].top5_predictions[:, test_id]
        for j in range(i + 1, num_networks):
            name_j = network_names[j]
            pred_j = network_predictions[name_j].top1_predictions[:, test_id]
            pred_j5 = network_predictions[name_j].top5_predictions[:, test_id]

            (corr_coeff, each_class) = get_all_classes(pred_i, pred_j)
            pairwise_comparison1[i, j] = corr_coeff
            pairwise_comparison1[j, i] = corr_coeff
            pairwise_comparison_classes1[i, j, :] = each_class
            pairwise_comparison_classes1[j, i, :] = each_class

            (corr_coeff, each_class) = get_all_classes(pred_i5, pred_j5)
            pairwise_comparison5[i, j] = corr_coeff
            pairwise_comparison5[j, i] = corr_coeff
            pairwise_comparison_classes5[i, j, :] = each_class
            pairwise_comparison_classes5[j, i, :] = each_class
    return (pairwise_comparison1, pairwise_comparison5, pairwise_comparison_classes1, pairwise_comparison_classes5)


def get_all_classes(pred_i, pred_j):
    all_classes = get_corr(pred_i, pred_j)
    each_class = [0] * num_classes
    for i in range(num_classes):
        si = int(class_inds[i, 0])
        ei = int(class_inds[i, 1])
        each_class[i] = get_corr(pred_i[si:ei], pred_j[si:ei])
    return (all_classes, each_class)

def get_corr(pred_i, pred_j):
    corr_coeff = np.sum(pred_i == pred_j) / pred_i.shape[0]
    return corr_coeff  

In [37]:
pairwise_comparison_classes1.shape

(2, 2, 1000)

In [39]:
np.savetxt('arash.csv', pairwise_comparison_classes1[0,1,:].flatten(), delimiter=',')

In [40]:
np.argmin(pairwise_comparison_classes1[0,1,:].flatten())

413

In [41]:
pairwise_comparison_classes1[0,1,:].flatten().min()

0.56